# ST-GCN for international trade

In [1]:
# Wheter or not we are executing on Google Colab
drive = False

# Only needed it the code is executed on Colab
if drive:
    from google.colab import drive
    drive.mount('/content/drive')
    !pip install tabulate

import import_ipynb
import dist_matrix

importing Jupyter notebook from dist_matrix.ipynb


## Packages

In [2]:
import random
import torch
import copy
import os
import json
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from tabulate import tabulate
import time
import calendar
from openpyxl import load_workbook

## Load data

In [3]:
# Loads data from a CSV file, removing the 'year' column and splitting data into train, validation and test
def load_data(file_path, len_train, len_val):
    df = pd.read_csv(file_path, header=0).drop(['year'], axis=1).values.astype(float)
    train = df[:len_train]
    val = df[len_train: len_train + len_val]
    test = df[len_train + len_val:]
    return train, val, test

# Transforms data to prepare input (x) and output (y) to be fed to the model
def data_transform(data, n_his, device):
    """
    Transforms data to prepare input (x) and output (y) to be fed to the model.

    Args:
    data: Data to be manipulated.
    n_his (int): Number of past timesteps.
    device: The device in which data will be stored.

    Returns:
    Torch.Tensor, Torch.Tensor: The input and output data.
    """
    n_year = data.shape[0]
    # The number of time series (in our setting, the number of countries)
    n_series = data.shape[1]
    # The number of time windows
    n_slot = n_year - n_his

    x = np.zeros([n_slot, 1, n_his, n_series])
    y = np.zeros([n_slot, n_series])
    for i in range(n_slot):
        x[i, :, :, :] = data[i:i + n_his].reshape(1, n_his, n_series)
        y[i] = data[i + n_his]
    return torch.Tensor(x).to(device), torch.Tensor(y).to(device)

## STGCN model

Architecture of the STGCN model, adapted to our dataset.

In [4]:
# stgcn

import math
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F

class align(nn.Module):
    def __init__(self, c_in, c_out):
        super(align, self).__init__()
        self.c_in = c_in
        self.c_out = c_out
        if c_in > c_out:
            self.conv1x1 = nn.Conv2d(c_in, c_out, 1)

    def forward(self, x):
        if self.c_in > self.c_out:
            return self.conv1x1(x)
        if self.c_in < self.c_out:
            return F.pad(x, [0, 0, 0, 0, 0, self.c_out - self.c_in, 0, 0])
        return x

class temporal_conv_layer(nn.Module):
    def __init__(self, kt, c_in, c_out, act="relu"):
        super(temporal_conv_layer, self).__init__()
        self.kt = kt
        self.act = act
        self.c_out = c_out
        self.align = align(c_in, c_out)
        if self.act == "GLU":
            self.conv = nn.Conv2d(c_in, c_out * 2, (kt, 1), 1)
        else:
            self.conv = nn.Conv2d(c_in, c_out, (kt, 1), 1)

    def forward(self, x):
        x_in = self.align(x)[:, :, self.kt - 1:, :]
        if self.act == "GLU":
            x_conv = self.conv(x)
            return (x_conv[:, :self.c_out, :, :] + x_in) * torch.sigmoid(x_conv[:, self.c_out:, :, :])
        if self.act == "sigmoid":
            return torch.sigmoid(self.conv(x) + x_in)
        return torch.relu(self.conv(x) + x_in)

class spatio_conv_layer(nn.Module):
    def __init__(self, ks, c, Lk):
        super(spatio_conv_layer, self).__init__()
        self.Lk = Lk
        self.theta = nn.Parameter(torch.FloatTensor(c, c, ks))
        self.b = nn.Parameter(torch.FloatTensor(1, c, 1, 1))
        self.reset_parameters()

    def reset_parameters(self):
        init.kaiming_uniform_(self.theta, a=math.sqrt(5))
        fan_in, _ = init._calculate_fan_in_and_fan_out(self.theta)
        bound = 1 / math.sqrt(fan_in)
        init.uniform_(self.b, -bound, bound)

    def forward(self, x):
        x_c = torch.einsum("knm,bitm->bitkn", self.Lk, x)
        x_gc = torch.einsum("iok,bitkn->botn", self.theta, x_c) + self.b
        return torch.relu(x_gc + x)

class st_conv_block(nn.Module):
    def __init__(self, ks, kt, n, c, p, Lk):
        super(st_conv_block, self).__init__()
        self.tconv1 = temporal_conv_layer(kt, c[0], c[1], "GLU")
        self.sconv = spatio_conv_layer(ks, c[1], Lk)
        self.tconv2 = temporal_conv_layer(kt, c[1], c[2])
        self.ln = nn.LayerNorm([n, c[2]])
        self.dropout = nn.Dropout(p)

    def forward(self, x):
        x_t1 = self.tconv1(x)
        x_s = self.sconv(x_t1)
        x_t2 = self.tconv2(x_s)
        x_ln = self.ln(x_t2.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)
        return self.dropout(x_ln)

class fully_conv_layer(nn.Module):
    def __init__(self, c):
        super(fully_conv_layer, self).__init__()
        self.conv = nn.Conv2d(c, 1, 1)

    def forward(self, x):
        return self.conv(x)

class output_layer(nn.Module):
    def __init__(self, c, T, n):
        super(output_layer, self).__init__()
        self.tconv1 = temporal_conv_layer(T, c, c, "GLU")
        self.ln = nn.LayerNorm([n, c])
        self.tconv2 = temporal_conv_layer(1, c, c, "sigmoid")
        self.fc = fully_conv_layer(c)

    def forward(self, x):
        x_t1 = self.tconv1(x)
        x_ln = self.ln(x_t1.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)
        x_t2 = self.tconv2(x_ln)
        return self.fc(x_t2)

class STGCN(nn.Module):
    def __init__(self, ks, kt, bs, T, n, Lk, p):
        super(STGCN, self).__init__()
        self.st_conv1 = st_conv_block(ks, kt, n, bs[0], p, Lk)
        self.st_conv2 = st_conv_block(ks, kt, n, bs[1], p, Lk)
        self.output = output_layer(bs[1][2], T - 4 * (kt - 1), n)

    def forward(self, x):
        x_st1 = self.st_conv1(x)
        x_st2 = self.st_conv2(x_st1)
        return self.output(x_st2)


## Utils

In [5]:
# Evaluates the model on a data set
def evaluate_model(model, loss, data_iter):
    """
    Evaluates the model on a data set.

    Args:
    model (torch.nn.Module): Model to be evaluated.
    loss (function): Loss function.
    data_iter (iterable): Data iterator.

    Returns:
    float: Average loss.
    """
    model.eval() # Setting the model in evaluation mode
    l_sum, n = 0.0, 0

    with torch.no_grad(): # Turning off gradients calculus
        for x, y in data_iter:
            y_pred = model(x).view(len(x), -1)  # Model prediction
            l = loss(y_pred, y) # Loss
            l_sum += l.item() * y.shape[0] # Accumulating the loss
            n += y.shape[0] # Number of examples

    return l_sum / n # Average loss

# Evaluates the model wrt MAE, MAPE and RMSE 
def evaluate_metric(model, data_iter, scaler):
    """
    Evaluates the model wrt MAE, MAPE and RMSE.

    Args:
    model (torch.nn.Module): Model to be evaluated.
    data_iter (iterable): Data iterator.
    scaler (scaler object): Objet to de-scale data.

    Returns:
    tuple: MAE, MAPE and RMSE metrics.
    """
    model.eval() # Setting the model in evaluation mode

    with torch.no_grad(): # Turning off gradients calculus
        mae, mape, mse = [], [], []

        for x, y in data_iter:
            # De-scale ground truth and predictions
            y = scaler.inverse_transform(y.cpu().numpy()).reshape(-1)
            y_pred = scaler.inverse_transform(model(x).view(len(x), -1).cpu().numpy()).reshape(-1)

            d = np.abs(y - y_pred) # Absolute error
            mae += d.tolist() # Accumulating MAE
            mape += (d / y).tolist() # Accumulating MAE
            mse += (d ** 2).tolist() # Accumulating MAE

        # Computing final metrics
        MAE = np.array(mae).mean() # Mean absolute error
        MAPE = np.array(mape).mean() * 100 # Mean percentage absolute error
        RMSE = np.sqrt(np.array(mse).mean()) # Square root of mean quadratic error
        
        return MAE, MAPE, RMSE

# For each experiment, a line is written to a local Excel file that contains the main figures:
# Type of architecture, number of nodes, hyper-parameters, training time, accuracy, hits@k, MRR...
def write_line_to_xlsx(output):
    current_GMT = time.gmtime()
    time_stamp = calendar.timegm(current_GMT)
    time_stamp = datetime.utcfromtimestamp(time_stamp).strftime('%Y-%m-%d %H:%M:%S')
    records = [tuple([time_stamp] + list(output.values()))]

    wb = load_workbook(xlsx_path)
    # Select First Worksheet
    ws = wb.worksheets[0]

    for record in records:
        # Append Row Values
        ws.append(record)

    wb.save(xlsx_path)

## Random Seed

In [7]:
torch.manual_seed(2333)
torch.cuda.manual_seed(2333)
np.random.seed(2333)
random.seed(2333)
torch.backends.cudnn.deterministic = True

## Device

In [6]:
# GPU is selected whenever available
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    # This is for M-series-based Macs
    if torch.backends.mps.is_available():
        device = torch.device("mps")
    else:
        device = torch.device("cpu")
device

device(type='mps')

## File Paths

In [7]:
from datetime import datetime

# date-based ID to distinguish models
def generate_id():
    now = datetime.now()
    id_str = now.strftime("%Y%m%d%H%M%S%f") # Formatting date and time with microseconds
    return id_str

# File paths for data   

# The matrix with distances between countries
kms_matrix_filename = "kms_distance.csv"
# The temporal series data
data_filename = "time_series.csv"
# File where the model will be saved
save_filename = f'model-{generate_id()[:10]}.pth'
# File where results will be saved
xlsx_filename = "tmp_results.xlsx"

main_dir = '/content/drive/My Drive/Colab Notebooks/MUIA/TFM_AlbertoTomasMartin/' if drive else os.getcwd()

input_dir = os.path.join(main_dir, "input")
output_dir = os.path.join(main_dir, "output")

kms_matrix_path = os.path.join(input_dir,"matrix",kms_matrix_filename)
data_path = os.path.join(input_dir,data_filename)
save_path = os.path.join(output_dir,save_filename)
xlsx_path = os.path.join(output_dir,xlsx_filename)

dist_matrix_dict = {
    "Kms": kms_matrix_path
}


#### Initial Data Presentation

In [ ]:
# Debug only

# TO-DO: what is this, exactly?
# Is this just the ECI by country by year? In this case, the dimensionality given below (probably taken from Marcos' TFM) is wrong

# [N years] x [N countries x N countries]
data_df = pd.read_csv(data_path)
data_df

 #### SITC Original Data

In [8]:
# This data can be dowloaded from https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/H8SFD2
stata_file_path = os.path.join(input_dir, 'SITC/country_partner_sitcproduct2digit_year.dta')
df = pd.read_stata(stata_file_path)

In [ ]:
# Debug only
print(df.head())
print(f"\nThe SITC dataset has {len(df)} rows and {len(df.columns)} columns")

In [ ]:
# Debug only
df[df.location_code=="AFG"][df.year==2018]

## Parameters

In [9]:
# From a total of 60 years (timesteps) of data, we split into train, validation and test.
# This means that
# - the first n_train years are used for training
# - the following n_val years are used for validation
# - the rest are used for test
# 
# Given each of these splits, data are generated as follows: a slice of n_his+1 consecutive years is taken, from which the first n_his are x, and the last one is y (values to be predicted)
#
# Example with n_train=35, n_his=10: since the training years span from 1962 to 1996 (corresponding to 35 timesteps), we can take 25 slices of 10+1=11 years [1962..1972], [1963..1973], .., [1986..1996].git.
# For each slice, the first 10 years are x, and the last year will be y

# n_his = 10 # Number of timesteps to be considered as input

# Conditions:
# - n_train+n_val+n_test = total number of years (timesteps) in the dataset
# - n_train >= n_his+1
# - n_val >= n_his+1
# - n_test >= n_his+1
#n_train = 35 # Number of examples at training
#n_val = 12 # Number of examples at validation
#n_test = 13 # Number of examples at test

# Ks, Kt = 4, 3 # Size of the kernel for the spatial and the temporal convolution, respectively

blocks = [[1, 512, 1024], [1024, 1024, 2048]] # Structure of blocks for the temporal and spatial convolution:
# The first list corresponds to input and output channels of the first two STConv blocks
# The second list corresponds to input and output channels of the third STConv block

drop_prob = 0 # Dropout probability (used to regularize the model and avoid overfitting)

# Training parameters
batch_size = 128
# epochs = 50
# lr = 1e-2

## Building the graph

We explore several option for distance, starting from the original matrix on the combination of geographical distance between the main cities of two countries (the ECI is computed using the SITC product data from 1962 to 2021, while the adjacency matrix considers bilateral distances between major cities weighted by population shares).

In [10]:
# Debug only
option = "Reciprocal" # ["Raw", "Random", "Constant", "Laplacian", "Reciprocal", "Exponential reciprocal"]

# Training & Saving the Model

## Data Loading

### Standardization

In [11]:
# Initializing a standard scaler to normalize data
scaler = StandardScaler()

In [12]:
def get_input_data(n_train, n_val, n_test, n_his):
    # Loading data from file and splitting into train, validation and test
    train_df, val_df, test_df = load_data(data_path, n_train , n_val)

    train = scaler.fit_transform(train_df)
    val = scaler.transform(val_df)
    test = scaler.transform(test_df)

    x_train, y_train = data_transform(train, n_his, device)
    x_val, y_val = data_transform(val, n_his, device)
    x_test, y_test = data_transform(test, n_his, device)

    # Creating a PyTorch DataLoader for train data, that allows iterating on batches
    train_data = torch.utils.data.TensorDataset(x_train, y_train)
    train_iter = torch.utils.data.DataLoader(train_data, batch_size, shuffle=False)
    val_data = torch.utils.data.TensorDataset(x_val, y_val)
    val_iter = torch.utils.data.DataLoader(val_data, batch_size, shuffle=False)
    test_data = torch.utils.data.TensorDataset(x_test, y_test)
    test_iter = torch.utils.data.DataLoader(test_data, batch_size, shuffle=False)

    return train_iter, val_iter, test_iter


In [ ]:
# Debug only
print(train.shape)
print(val.shape)
print(test.shape)

In [ ]:
# Debug only
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)
print(x_test.shape)
print(y_test.shape)

## Loss & Model & Optimizer

In [13]:
# Loss function: MSE
loss = nn.MSELoss()


In [14]:
def get_optimizer(optimizer_index, model, lr):
    # Defining optimizer from a dictionary
    optimizers = {
        "RMSprop" : torch.optim.RMSprop(model.parameters(), lr=lr), # Optimization using RMSprop wrt the specified learning rate
        "AdamW" : torch.optim.AdamW(model.parameters(), lr=lr)    # Optimization using AdamW wrt the specified learning rate
    }
    return optimizers[optimizer_index]

def get_scheduler(scheduler_index, optimizer, epochs):
    # Scheduler dictionary
    schedulers = {
        # Reduces the learning rate by gamma every step_size epochs
        "StepLR": torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.6),
        # Reduces the learning rate exponencially on gamma
        "ExponentialLR": torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99),
        # Adjusts the learning rate following a cosine function, with minimum eta_min
        "CosineAnnealingLR": torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=0)
    }
    return schedulers[scheduler_index]


## Training

In [15]:
def train(model,epochs,train_iter,val_iter):
    patience = 4

    # Initializing variables
    min_train_loss = np.inf # Best observed training loss
    min_val_loss = np.inf # Best observed validation loss
    best_epoch_train = 0 # Epoch when the best training loss is observed
    best_epoch_val = 0 # Epoch when the best validation loss is observed
    train_loss_list, val_loss_list = [], [] # Lists to store traininng and validation loss through epochs

    # Initializing lists
    last_val_losses = [np.inf] * patience # Current validation losses for comparison

    # Initializing the model state
    best_model_state = copy.deepcopy(model.state_dict())

    print(f"Training for {epochs} epochs")
    # Main training loop
    for epoch in range(1, epochs + 1):
        l_sum = 0.0 # Initializing loss sum
        n = 0
        model.train()

        # Loop on training data
        for x, y in train_iter:
            y_pred = model(x).view(len(x), -1) # Model prediction
            l = loss(y_pred, y) # Loss
            optimizer.zero_grad() # Clean gradients
            l.backward() # Back-propagation
            optimizer.step() # Update parameters
            l_sum += l.item() * y.shape[0] # Accumulating loss
            n += y.shape[0] # Number of examples so far # TO-DO: is 24 a meaningful number?

        scheduler.step() # Update learning rate according to the scheduler

        # Evaluating the model on the validation set
        val_loss = evaluate_model(model, loss, val_iter)

        train_loss = l_sum / n # Average training loss
        train_loss_list.append(train_loss)
        val_loss_list.append(val_loss)

        # Possibly updating the best training loss
        if train_loss < min_train_loss:
            min_train_loss = train_loss
            best_epoch_train = epoch

        if epoch >= patience:
            # Keeping the last validation losses
            for i in reversed(1,range(patience)):
                last_val_losses[i] = last_val_losses[i-1]
            last_val_losses[0] = val_loss

            if all(min_val_loss < c for c in last_val_losses):
                print(f"Early stopping at epoch {epoch}")
                break
            else:
                min_val_loss = min(last_val_losses)
                best_epoch_val = epoch
                best_model_state = copy.deepcopy(model.state_dict())

            # Possibly updating the best validation losses and saving the model state
            #if all(c < b for c, b in zip(last_val_losses, best_val_losses)):
            #    best_val_losses = last_val_losses.copy()
            #    min_val_loss = val_loss
            #    best_epoch_val = epoch
            #    best_model_state = copy.deepcopy(model.state_dict())

        # Printing the current learning rate and losses, the first epoch and after each 10 epochs
        if epoch % 10 == 0:
            current_lr = optimizer.param_groups[0]['lr']
            print(f"Epoch {epoch} -> Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, Learning Rate: {current_lr:.6f}")

    # After training is complete, loading the best model state
    model.load_state_dict(best_model_state)

    return min_train_loss, best_epoch_train, min_val_loss, best_epoch_val

# Evaluation

In [16]:
def test(model,test_iter):
    l = evaluate_model(model, loss, test_iter)
    MAE, MAPE, RMSE = evaluate_metric(model, test_iter, scaler)
    return l, MAE, MAPE, RMSE

## Experiment driver

This is to run experiments with different values for hyperparameters, and store the results in a file

In [17]:
options = {}

# The type of adjacency matrix
options['adjacency_matrix'] = ["Reciprocal"] # ["Raw", "Random", "Constant", "Reciprocal", "Exponential reciprocal", "Laplacian"]
# The degree of kernels
options['spatial_kernel_size'] = [4]
options['temporal_kernel_size'] = [3]
# The number of epochs
options['n_epochs'] = [20]
# The (initial) learning rate
options['learning_rate'] = [0.01]
# History length (the number of consecutive timestep from which the model learns)
options['history_length'] = [10]
# The size (in years or timesteps) of train, validation and test sets
options['splits_size'] = [(35,12,13)]

number_of_runs = 1 # This is used to run the same test several times

n = 0
for _ in options['adjacency_matrix']:
    for _ in options['spatial_kernel_size']:
        for _ in options['temporal_kernel_size']:
            for _ in options['n_epochs']:
                for _ in options['learning_rate']:
                    for _ in options['history_length']:
                        for _ in options['splits_size']:
                            n = n+1
print(f'A total number of {n} learning tasks will be executed {number_of_runs} times')

A total number of 1 learning tasks will be executed 1 times


In [18]:
n = 0

this_run = 0
while (this_run<number_of_runs):
    for adjacency_matrix in options['adjacency_matrix']:
        for spatial_kernel_size in options['spatial_kernel_size']:
            for temporal_kernel_size in options['temporal_kernel_size']:
                for n_epochs in options['n_epochs']:
                    for learning_rate in options['learning_rate']:
                        for history_length in options['history_length']:
                            # TO-DO: This is unused for the moment (train_iter, val_iter and test_iter have been computed before)
                            for splits_size in options['splits_size']:
                                # TO-DO: improve this by only specifying the kind of distance we want (to indicate the path should be redundant)
                                A = dist_matrix.get_adjacency_matrix(adjacency_matrix,dist_matrix_dict,"Kms",spatial_kernel_size)
                                A = A.to(device)
                                train_iter, val_iter, test_iter = get_input_data(splits_size[0], splits_size[1], splits_size[2], history_length)
                                n_features = A.shape[1]
                                # Parameters:
                                # - ks: Size of the spatial kernel
                                # - kt: Size of the temporal kernel
                                # - bs: Lists of channels for each block
                                # - T: Number of timesteps
                                # - n: Number of time intervals (slots)
                                # - Lk: Chebyshev polynomials
                                # - p: Dropout probability
                                model = STGCN(spatial_kernel_size, temporal_kernel_size, blocks, history_length, n_features, A, drop_prob).to(device)
                                optimizer = get_optimizer("RMSprop",model,learning_rate)
                                scheduler = get_scheduler("CosineAnnealingLR",optimizer,n_epochs)
                                print("\nSTARTING TRAINING\n")
                                start_time = time.time()
                                min_train_loss, best_epoch_train, min_val_loss, best_epoch_val = train(model,n_epochs,train_iter,val_iter)
                                end_time = time.time()
                                train_time = end_time-start_time
                                print(f"Best Train Loss: {min_train_loss:.4f} at epoch {best_epoch_train}")
                                print(f"Best Validation Loss: {min_val_loss:.4f} at epoch {best_epoch_val}")
                                print(f"Training time: {train_time} s")
                                print("\nSTARTING TESTING\n")
                                test_loss, MAE, MAPE, RMSE = test(model,test_iter)
                                print( f"Test loss: {test_loss:.4f}")
                                print( f"MAE: {MAE:.4f}, MAPE: {MAPE:.2f}%, RMSE: {RMSE:.4f}")
                                output = {
                                    # params
                                    "adjacency_matrix": adjacency_matrix,
                                    "spatial_kernel_size": spatial_kernel_size,
                                    "temporal_kernel_size": temporal_kernel_size,
                                    "n_epochs": n_epochs,
                                    "learning_rate": learning_rate,
                                    "history_length": history_length,
                                    "train_size": splits_size[0],
                                    "val_size": splits_size[1],
                                    "test_size": splits_size[2],
                                    # results
                                    "min_train_loss": min_train_loss,
                                    "best_epoch_train": best_epoch_train,
                                    "min_val_loss": min_val_loss,
                                    "best_epoch_val": best_epoch_val,
                                    "train_time": train_time,
                                    "test_loss": test_loss,
                                    "MAE": MAE,
                                    "MAPE": MAPE,
                                    "RMSE": RMSE
                                } 
                                write_line_to_xlsx(output)
                                print(f"### Learning task #{n+1} done\n\n")
                                n = n+1
    this_run = this_run+1
print(f'Executed {n} learning tasks')

Computing the reciprocal

STARTING TRAINING

Training for 20 epochs


/Users/damiano/Library/Python/3.9/lib/python/site-packages/torch/nn/functional.py:4552: UserWarning: MPS: The constant padding of more than 3 dimensions is not currently supported natively. It uses View Ops default implementation to run. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/Pad.mm:472.)
  return torch._C._nn.pad(input, pad, mode, value)


IndexError: list assignment index out of range

# END OF MY CODE

## Visualization

#### Filter

In [ ]:
def filter_lists(list1, list2, limit, min_index=0, max_index=100 ):
    list1 = list1.copy()
    list2 = list2.copy()

    indices_to_be_removed = []

    # Identifying the indices to be removed after the first pass
    for index, (value1, value2) in enumerate(zip(list1, list2)):
        if value1 >= limit or value2 >= limit:
            indices_to_be_removed.append(index)

    # Removing indices in reversed order
    for index in sorted(indices_to_be_removed, reverse=True):
        list1.pop(index)
        list2.pop(index)

    # Removing all elements after max_index
    if max_index < len(list1):
        del list1[max_index+1:]
        del list2[max_index+1:]

    # Removing all elements before min_index
    if min_index > 0:
        list1 = list1[min_index:]
        list2 = list2[min_index:]

    return list1, list2

def filter_list(list, limit, min_index=0, max_index=100):
    list = list.copy()

    indices_to_be_removed = []

    # Identifying the indices to be removed after the first pass
    for index, value in enumerate(list):
        if value >= limit:
            indices_to_be_removed.append(index)

    # Removing indices in reversed order
    for index in sorted(indices_to_be_removed, reverse=True):
        list.pop(index)

    # Removing all elements after max_index
    if max_index < len(list):
        del list[max_index+1:]

    # Removing all elements before min_index
    if min_index > 0:
        list = list[min_index:]

    return list

#### Showing losses

In [ ]:
# This is to show only part of the results in a chart

# Parameter configuration for loss filtering
# 'filter' indicates whether filters have to be applied to the lists of losses
filter = True
# Parameters specify the epoch range and the maximum value for a loss to be kept
max_value = 50
min_epoch = 60
max_epoch = 200

if filter:
    # Applying the filter to obtain loss lists that fit the constraints
    train_loss_list_fil, val_loss_list_fil = filter_lists(train_loss_list, val_loss_list, max_value, min_epoch, max_epoch)
    epochs_range = range(min_epoch + 1, min_epoch + 1 + len(train_loss_list_fil))
else:
    # If filtering is off, use the original loss lists
    train_loss_list_fil, val_loss_list_fil = train_loss_list, val_loss_list
    epochs_range = range(1, len(train_loss_list_fil) + 1)

# Creating the graph
plt.figure(figsize=(10, 6))
plt.plot(epochs_range, train_loss_list_fil, label='Train Loss', marker='o')
plt.plot(epochs_range, val_loss_list_fil, label='Validation Loss', marker='o')
# Adding title and labels
plt.title('Train and Validation Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

## Save model

In [ ]:
if (drive):
    save_model_path = f'/content/drive/My Drive/TFM/models/model-{generate_id()[:12]}.pth'
    results_path = f'/content/drive/My Drive/TFM/models/model-{generate_id()[:12]}.pth'
else:
    save_model_path = os.path.join(main_dir, "models", f'model-{generate_id()[:12]}.pth')
    results_path = os.path.join(main_dir, "results", f'results-{generate_id()[:12]}.json')

print(save_model_path)
print(results_path)

In [ ]:
#torch.save(model.state_dict(), save_model_path)

# Creating a dictionary with lists
data = {
    'train_loss_list': train_loss_list,
    'val_loss_list': val_loss_list,
    'MAE': f"{MAE:.4f}",
    'MAPE': f"{MAPE:.2f}%",
    'RMSE': f"{RMSE:.4f}",
}

# Save to a JSON file
with open(results_path, 'w') as json_file:
    json.dump(data, json_file, indent=4)

# Load Best Model

In [ ]:
model_id = 202407032036
# 202407012024

drive = False
if (drive):
    load_path = f'/content/drive/My Drive/TFM/models/model-{model_id}.pth'
else:
    load_path = os.path.join(main_dir, "models", f'model-{model_id}.pth')

print(load_path)

model = STGCN(Ks, Kt, blocks, n_his, n_features, Lk, drop_prob).to(device)
state_dict = torch.load(load_path)
model.load_state_dict(state_dict)
type(model)

# Prediction

In [ ]:
model.eval()

# (back-propagation is not needed here)
with torch.no_grad():
    # Inicializing lists to store error metrics
    mae, mape, mse = [], [], []

    # Iterating on training data
    for x, y in test_iter:
        x_raw = x # Store original input data
        y_raw = y # Store original labels

        # Transforming ground labels to their original scale
        y = scaler.inverse_transform(y.cpu().numpy()).reshape(-1)

        # Running model prediction and transforming to original scale
        y_pred = scaler.inverse_transform(model(x).view(len(x), -1).cpu().numpy()).reshape(-1)

In [ ]:
# Returns the index and column where the value is found
def search_value_in_dataframe(df, value):
    result = []  

    for col in df.columns:
        matching_rows = df[df[col] == value].index.tolist()

        for row in matching_rows:
            result.append((row, col))

    return result  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Ground and predicted values
data1 = y
data2 = y_pred

# Creating a figure
plt.figure(figsize=(16, 6))

# Showing the first data set
plt.plot(data1, label='Data 1', color='b')

# Showing the second data set over the first
plt.plot(data2, label='Data 2', color='r')

# Add title and labels
plt.title('Overlapping of two data sets')
plt.xlabel('Index')
plt.ylabel('Value')

plt.legend()

plt.show()

In [ ]:
len(y)

### Predictions for 2020

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Selecting a year (131 countries, columns) for input data and predictions
data1 = y[:130] 
data2 = y_pred[:130]  

# Creating a figure
plt.figure(figsize=(16, 6))

# Defining the index for bars
indices = np.arange(len(data1))

# Bar width
width = 0.4

# Show the first data set as bars
plt.bar(indices - width/2, data1, width=width, label='Data 1', color='blue')

# Show the first data set as bars
plt.bar(indices + width/2, data2, width=width, label='Data 2', color='red')

plt.title('Comparing two data sets')
plt.xlabel('Index')
plt.ylabel('Value')

plt.legend()

plt.show()

In [ ]:
list(matrix_df.columns)[2]

In [ ]:
list(matrix_df.columns)[61]

In [ ]:
# Calcular la diferencia absoluta entre los valores correspondientes
diferencias = np.abs(data1[:130] - data2[:130])

# Encontrar la posición con la mayor diferencia
posicion_max_diferencia = np.argmax(diferencias)
mayor_diferencia = diferencias[posicion_max_diferencia]

print(f"La posición con la mayor diferencia es: {posicion_max_diferencia}")
print(f"La mayor diferencia es: {mayor_diferencia}")
varianza = np.var(data_df['ALB'])
varianza

In [ ]:
type(data_df['AUS'])

plt.figure(figsize=(10, 6))
plt.plot(data_df['AUS'], label='Series Data')
plt.xlabel('Índice')
plt.ylabel('Valor')
plt.title('Gráfica de pandas.core.series.Series')
plt.legend()

# Mostrar la gráfica
plt.show()

In [ ]:
# Calcular la diferencia absoluta entre los valores correspondientes
diferencias = np.abs(data1 - data2)

# Encontrar la posición con la mayor diferencia
posicion_max_diferencia = np.argmax(diferencias)
mayor_diferencia = diferencias[posicion_max_diferencia]

print(f"El pais con la mayor diferencia es: {list(matrix_df.columns)[posicion_max_diferencia]}")
print(f"La mayor diferencia es: {mayor_diferencia}")


In [ ]:
type(data_df['IRQ'])

plt.figure(figsize=(10, 6))
plt.plot(data_df['IRQ'], label='Series Data')
plt.xlabel('Índice')
plt.ylabel('Valor')
plt.title('Gráfica de pandas.core.series.Series')
plt.legend()

# Mostrar la gráfica
plt.show()

In [ ]:
data_df = pd.read_csv(data_path)

In [ ]:
data = data.drop(columns=['year'])

In [ ]:
y_1 = y[:131]
y_2 = y[131:]

y_pred_1 = y_pred[:131]
y_pred_2 = y_pred[:131]

In [ ]:
results_2020 = pd.DataFrame([y_1, y_pred_1], columns=data.columns)
results_2021 = pd.DataFrame([y_2, y_pred_2], columns=data.columns)

In [ ]:
fila_1 = results_2020.iloc[0]  # Primera fila
fila_2 = results_2020.iloc[1]  # Segunda fila

# Configurar los valores de las dos filas
valores_fila_1 = fila_1.values
valores_fila_2 = fila_2.values

labels = fila_1.index

# Crear la gráfica de barras
x = range(len(labels))
width = 0.4  # Ancho de las barras

fig, ax = plt.subplots(figsize=(10, 6))  # Ajustar el tamaño de la figura si es necesario
ax.bar(x, valores_fila_1, width=width, label='Fila 1', align='center')
ax.bar([p + width for p in x], valores_fila_2, width=width, label='Fila 2', align='center')

# Añadir etiquetas y título
ax.set_xlabel('Columnas')
ax.set_ylabel('Valores')
ax.set_title('Comparación de Valores entre dos Filas')
ax.set_xticks([p + width / 2 for p in x])
ax.set_xticklabels([''] * len(labels))  # Configurar las etiquetas del eje x como vacías
ax.legend()

# Mostrar la gráfica
plt.show()


### predicts 2021

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Supongamos que tienes dos numpy arrays de ejemplo
data1 = y[131:]  # Ejemplo de datos aleatorios
data2 = y_pred[131:]  # Ejemplo de datos aleatorios

# Crear una figura y un conjunto de ejes
plt.figure(figsize=(16, 6))

# Definir el índice para las barras
indices = np.arange(len(data1))

# Ancho de las barras
width = 0.4

# Graficar el primer conjunto de datos como barras
plt.bar(indices - width/2, data1, width=width, label='Data 1', color='b')

# Graficar el segundo conjunto de datos como barras
plt.bar(indices + width/2, data2, width=width, label='Data 2', color='r')

# Añadir título y etiquetas
plt.title('Comparación de dos conjuntos de datos')
plt.xlabel('Índice')
plt.ylabel('Valor')

# Añadir una leyenda
plt.legend()

# Mostrar el gráfico
plt.show()


In [ ]:
list(matrix_df.columns)[4]

In [ ]:
data1 = y[131:]
data2 = y_pred[131:]

In [ ]:
# Calcular la diferencia absoluta entre los valores correspondientes
diferencias = np.abs(data1 - data2)

# Encontrar la posición con la mayor diferencia
posicion_max_diferencia = np.argmax(diferencias)
mayor_diferencia = diferencias[posicion_max_diferencia]

print(f"El pais con la mayor diferencia es: {list(matrix_df.columns)[posicion_max_diferencia]}")
print(f"La mayor diferencia es: {mayor_diferencia}")
print( f"Varianza de {list(matrix_df.columns)[posicion_max_diferencia]} : {np.var(data_df[list(matrix_df.columns)[posicion_max_diferencia]])}")

In [ ]:
varianza = np.var(data_df[list(matrix_df.columns)[posicion_max_diferencia]])
print( f"Varianza de {list(matrix_df.columns)[posicion_max_diferencia]} : {}")

In [ ]:
plt.figure(figsize=(16, 10))
plt.plot(data_df['ALB'], label='Series Data')
plt.xlabel('Years')
plt.ylabel('SITC ECI')
plt.title('SITC ECI of Albania')
plt.legend()

# Mostrar la gráfica
plt.show()

In [ ]:
# Calcular la diferencia absoluta entre los valores correspondientes
diferencias = np.abs(data1 - data2)

# Encontrar la posición con la mayor diferencia
posicion_max_diferencia = np.argmin(diferencias)
mayor_diferencia = diferencias[posicion_max_diferencia]

print(f"El pais con la menor diferencia es: {list(matrix_df.columns)[posicion_max_diferencia]}")
print(f"La menor diferencia es: {mayor_diferencia}")
print( f"Varianza de {list(matrix_df.columns)[posicion_max_diferencia]} : {np.var(data_df[list(matrix_df.columns)[posicion_max_diferencia]])}")

In [ ]:
plt.figure(figsize=(16, 10))
plt.plot(data_df['PHL'], label='Series Data')
plt.xlabel('Years')
plt.ylabel('SITC ECI')
plt.title('SITC ECI of The Philippines')
plt.legend()

# Mostrar la gráfica
plt.show()

In [ ]:
data_df = pd.read_csv(data_path)

In [ ]:
data = data.drop(columns=['year'])

In [ ]:
y_1 = y[:131]
y_2 = y[131:]

y_pred_1 = y_pred[:131]
y_pred_2 = y_pred[:131]

In [ ]:
results_2020 = pd.DataFrame([y_1, y_pred_1], columns=data.columns)
results_2021 = pd.DataFrame([y_2, y_pred_2], columns=data.columns)

In [ ]:
fila_1 = results_2020.iloc[0]  # Primera fila
fila_2 = results_2020.iloc[1]  # Segunda fila

# Configurar los valores de las dos filas
valores_fila_1 = fila_1.values
valores_fila_2 = fila_2.values

labels = fila_1.index

# Crear la gráfica de barras
x = range(len(labels))
width = 0.4  # Ancho de las barras

fig, ax = plt.subplots(figsize=(10, 6))  # Ajustar el tamaño de la figura si es necesario
ax.bar(x, valores_fila_1, width=width, label='Fila 1', align='center')
ax.bar([p + width for p in x], valores_fila_2, width=width, label='Fila 2', align='center')

# Añadir etiquetas y título
ax.set_xlabel('Columnas')
ax.set_ylabel('Valores')
ax.set_title('Comparación de Valores entre dos Filas')
ax.set_xticks([p + width / 2 for p in x])
ax.set_xticklabels([''] * len(labels))  # Configurar las etiquetas del eje x como vacías
ax.legend()

# Mostrar la gráfica
plt.show()


### countries study

In [ ]:
matrix_columns = list(matrix_df.columns)

In [ ]:
def n_closest_countries(df, country, N):
    distances = df[country]
    sorted_distances = distances.sort_values()
    closest_indices = sorted_distances.index[1:N+1]  # Skip the first one as it is the country itself
    closest_countries = [matrix_columns[i] for i in closest_indices]
    return closest_countries

def n_farthest_countries(df, country, N):
    distances = df[country]
    sorted_distances = distances.sort_values(ascending=False)
    farthest_indices = sorted_distances.index[:N]
    farthest_countries = [matrix_columns[i] for i in farthest_indices]
    return farthest_countries

def calcular_varianza(df, columna, previous_n=60):
    varianza = df[columna][:previous_n].var()
    return varianza


In [ ]:
def get_data_countries(country_name, N=5):
    # Calcular la varianza para cada uno de los países más cercanos y calcular la media de estas varianzas
    closest_countries = n_closest_countries(matrix_df, country_name, N)
    varianzas = [calcular_varianza(data_df, country, 10) for country in closest_countries]
    media_varianzas = np.mean(varianzas)

    print(f"Los {N} países más cercanos a {country_name} son: {closest_countries}")
    print(f"La varianza de cada uno de estos paíse de los ultimos 10 años es: {[round(v, 4) for v in varianzas]}")
    print(f"La media de las varianzas es: {round(media_varianzas, 4)}")

    # Calcular la varianza para cada uno de los países más cercanos y calcular la media de estas varianzas
    farthest_countries = n_farthest_countries(matrix_df, country_name, N)
    varianzas = [calcular_varianza(data_df, country, 10) for country in farthest_countries]
    media_varianzas = np.mean(varianzas)

    print(f"Los {N} países más lejanos a {country_name} son: {farthest_countries}")
    print(f"La varianza de cada uno de estos países de los ultimos 10 años es: {[round(v, 4) for v in varianzas]}")
    print(f"La media de las varianzas es: {round(media_varianzas, 4)}")

    return closest_countries, farthest_countries

def represent_countries(countries, previous_n=10):
  # Filtrar las columnas más cercanas
  filtered_df = data_df[countries][:previous_n]

  # Crear la gráfica
  plt.figure(figsize=(14, 8))

  # Graficar cada columna
  for column in filtered_df.columns:
      plt.plot(filtered_df.index, filtered_df[column], label=column)

  # Añadir título y etiquetas
  plt.title('Valores de las columnas más cercanas')
  plt.xlabel('Índice')
  plt.ylabel('Valor')

  # Añadir una leyenda
  plt.legend()

  # Mostrar la gráfica
  plt.show()

In [ ]:
closest_countries, farthest_countries = get_data_countries('ALB')

In [ ]:
represent_countries(farthest_countries)

# Comparación modelos

## Load results

In [ ]:
regular_data = "results-20240630"
inverted_matrix = "results-20240701"

results_main_path = '/mnt/e/ARCHIVOS/UNIVERSIDAD/MUIA/TFM/Development/stgcn-colab/results'

### List files

In [ ]:
import os

def list_files(carpeta, prefijo):
    # Lista para almacenar los paths de los archivos que cumplen con el criterio
    archivos_filtrados = []

    # Iterar sobre los archivos en la carpeta
    for archivo in os.listdir(carpeta):
        # Comprobar si el archivo comienza con el prefijo especificado
        if archivo.startswith(prefijo):
            # Crear el path completo del archivo y añadirlo a la lista
            path_completo = os.path.join(carpeta, archivo)
            archivos_filtrados.append(path_completo)

    return archivos_filtrados

result_list = list_files(results_main_path, regular_data)

# Imprimir los paths de los archivos que cumplen con el criterio
for result_file in result_list:
    print(result_file)


In [ ]:
results_dict = {
    "AdamW - CosineAnnealingLR": result_list[0],
    "RMSprop - CosineAnnealingLR": result_list[1],
    "AdamW -CosineAnnealingLR": result_list[2],
    "AdamW -CosineAnnealingLR": result_list[3],
}

### Enum files and search

In [ ]:
def leer_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

def leer_result(file_path):
    file_path = f'{results_main_path}/results-{file_path}.json'
    return leer_json(file_path)

def get_results_data(model_dict):
    for key, value in model_dict.items():
        # Leer el archivo JSON correspondiente usando el "id"
        json_data = leer_result(value["id"])

        # Añadir los datos del JSON al diccionario bajo la misma clave
        model_dict[key].update(json_data)

    return model_dict

In [ ]:
regular_data_dict = {
    0: {"name": "AdamW - CosineAnnealingLR", "id": 202407032033},
    1: {"name": "RMSprop - CosineAnnealingLR", "id": 202407042014},
    2: {"name": "RMSprop - StepLR", "id": 202407032036},
    3: {"name": "AdamW - StepLR", "id": 202407032034},
}

inverted_matrix_dict = {
    1: {"name": "RMSprop - CosineAnnealingLR", "id": 202407032058},
    2: {"name": "RMSprop - StepLR", "id": 202407032056},
}

exp_inverted_matrix_dict = {
    1: {"name": "RMSprop - CosineAnnealingLR", "id": 202407032106},
    2: {"name": "RMSprop - StepLR", "id": 202407032107},
}

## Grafico

In [ ]:
results_data = get_results_data(regular_data_dict)

regular_result_loss = leer_result('202407042014')['val_loss_list']
inverted_result_loss = leer_result('202407032058')['val_loss_list']
exp_inverted_result_loss = leer_result('202407032106')['val_loss_list']
exp_inverted_loss_expanded = leer_result('202407042053')['val_loss_list']
exp_inverted_loss_expanded_v2 = leer_result('202407042053')['val_loss_list']

### Listas de loss

In [ ]:
# Filtrar
    # True | False
filtrar = True
max_value = 3.5
min_epoch = 60
max_epoch = 200
# 'train_loss_list' | 'val_loss_list'
loss_list_name = 'val_loss_list'

# Crear la gráfica
plt.figure(figsize=(12, 8))

for key, value in results_data.items():
    loss_list = value[loss_list_name]

    if(filtrar):
        loss_list_fil = filtrar_valores_lista(loss_list, max_value, min_epoch, max_epoch)
        epochs_range = range(min_epoch + 1, min_epoch + 1 + len(loss_list_fil))
    else:
        loss_list_fil = loss_list
        epochs_range = range(1, len(loss_list_fil) + 1)

    name = value['name']
    plt.plot(epochs_range, loss_list_fil, label=name, marker='o')

# Añadir títulos y etiquetas
plt.title('Val Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

### Métricas

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Recopilar los valores de MAPE
model_names = []
mape_values = []

for key, value in results_data.items():
    model_names.append(value['name'])
    mape_values.append(float(value['MAPE'].strip('%')))

# Configurar la posición de las barras
x = np.arange(len(model_names))
width = 0.4  # Ancho de las barras

# Crear la gráfica
fig, ax = plt.subplots(figsize=(8, 6))

# Añadir las barras para MAPE
rects = ax.bar(x, mape_values, width, label='MAPE')

# Añadir etiquetas y títulos
ax.set_xlabel('Model')
ax.set_ylabel('MAPE (%)')
ax.set_title('Model MAPE Performance')
ax.set_xticks(x)
ax.set_xticklabels(model_names, rotation=45, ha='right')
ax.legend()

# Ajustar el límite superior del eje y para dar más margen
max_height = max(mape_values)
ax.set_ylim(0, max_height + 5)  # Añade un margen del 5% sobre el valor máximo

# Añadir los valores encima de las barras
def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(round(height, 2)),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

autolabel(rects)

# Mostrar la gráfica
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Recopilar los valores de MAE, MAPE y RMSE
model_names = []
mae_values = []
mape_values = []
rmse_values = []

for key, value in results_data.items():
    model_names.append(value['name'])
    mae_values.append(float(value['MAE']))
    rmse_values.append(float(value['RMSE']))

# Configurar la posición de las barras
x = np.arange(len(model_names))
width = 0.2  # Ancho de las barras

# Crear la gráfica
fig, ax = plt.subplots(figsize=(12, 8))

# Añadir las barras para MAE, MAPE y RMSE
rects1 = ax.bar(x - width, mae_values, width, label='MAE')
rects3 = ax.bar(x + width, rmse_values, width, label='RMSE')

# Añadir etiquetas y títulos
ax.set_xlabel('Model')
ax.set_ylabel('Scores')
ax.set_title('Model Performance Metrics')
ax.set_xticks(x)
ax.set_xticklabels(model_names, rotation=45, ha='right')
ax.legend()

# Añadir los valores encima de las barras
def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(round(height, 5)),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 5),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

autolabel(rects1)
autolabel(rects3)

# Mostrar la gráfica
plt.tight_layout()
plt.show()


### All data types

In [ ]:
# Parámetros de filtrado
# False | True
filtrar = True
max_value = 1000
min_epoch = 10
max_epoch = 150

# Filtrar las listas de resultados si es necesario
if filtrar:
    regular_result_loss_fil = filtrar_valores_lista(regular_result_loss, max_value, min_epoch, max_epoch)
    inverted_result_loss_fil = filtrar_valores_lista(inverted_result_loss, max_value, min_epoch, max_epoch)
    exp_inverted_result_loss_fil = filtrar_valores_lista(exp_inverted_result_loss, max_value, min_epoch, max_epoch)
else:
    regular_result_loss_fil = regular_result_loss
    inverted_result_loss_fil = inverted_result_loss
    exp_inverted_result_loss_fil = exp_inverted_result_loss

# Crear rangos de épocas correspondientes
epochs_range_regular = range(min_epoch + 1, min_epoch + 1 + len(regular_result_loss_fil))
# epochs_range_inverted = range(min_epoch + 1, min_epoch + 1 + len(inverted_result_loss_fil))
epochs_range_exp_inverted = range(min_epoch + 1, min_epoch + 1 + len(exp_inverted_result_loss_fil))

# Crear la gráfica
plt.figure(figsize=(12, 8))

# Plotear los resultados
plt.plot(epochs_range_regular, regular_result_loss_fil, label='Regular Loss', marker='o')
# plt.plot(epochs_range_inverted, inverted_result_loss_fil, label='Inverted Loss', marker='o')
plt.plot(epochs_range_exp_inverted, exp_inverted_result_loss_fil, label='Exp Inverted Loss', marker='o')

# Añadir títulos y etiquetas
plt.title('Validation Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Mostrar la gráfica
plt.tight_layout()
plt.show()

In [ ]:
exp_inverted_loss_expanded = exp_inverted_loss_expanded_fil
exp_inverted_loss_expanded_v2 = val_loss_list

In [ ]:
# False | True
filtrar = True
max_value = 300
min_epoch = 0
max_epoch = 10

# Filtrar las listas de resultados si es necesario
if filtrar:
    regular_result_loss_fil = filtrar_valores_lista(regular_result_loss, max_value, min_epoch, max_epoch)
    # inverted_result_loss_fil = filtrar_valores_lista(inverted_result_loss, max_value, min_epoch, max_epoch)
    exp_inverted_result_loss_fil = filtrar_valores_lista(exp_inverted_result_loss, max_value, min_epoch, max_epoch)
    exp_inverted_loss_expanded_fil = filtrar_valores_lista(exp_inverted_loss_expanded, max_value, min_epoch, max_epoch)
    # exp_inverted_loss_expanded_v2_fil = filtrar_valores_lista(exp_inverted_loss_expanded_v2, max_value, min_epoch, max_epoch)
else:
    regular_result_loss_fil = regular_result_loss
    # inverted_result_loss_fil = inverted_result_loss
    exp_inverted_result_loss_fil = exp_inverted_result_loss
    exp_inverted_loss_expanded_fil = exp_inverted_loss_expanded
    # exp_inverted_loss_expanded_v2_fil = exp_inverted_loss_expanded_v2

# Crear rangos de épocas correspondientes
epochs_range_regular = range(min_epoch + 1, min_epoch + 1 + len(regular_result_loss_fil))
# epochs_range_inverted = range(min_epoch + 1, min_epoch + 1 + len(inverted_result_loss_fil))
epochs_range_exp_inverted = range(min_epoch + 1, min_epoch + 1 + len(exp_inverted_result_loss_fil))
epochs_range_exp_inverted_expanded = range(min_epoch + 1, min_epoch + 1 + len(exp_inverted_loss_expanded_fil))
# epochs_range_exp_inverted_expanded_v2 = range(min_epoch + 1, min_epoch + 1 + len(exp_inverted_loss_expanded_v2_fil))

# Crear la gráfica
plt.figure(figsize=(12, 8))

# Plotear los resultados
plt.plot(epochs_range_regular, regular_result_loss_fil, label='Regular Loss', marker='o')
# plt.plot(epochs_range_inverted, inverted_result_loss_fil, label='Inverted Loss', marker='o')
plt.plot(epochs_range_exp_inverted, exp_inverted_result_loss_fil, label='Exp Inverted Loss', marker='o')
plt.plot(epochs_range_exp_inverted_expanded, exp_inverted_loss_expanded_fil, label='Exp Inverted Loss Expanded', marker='o')
# plt.plot(epochs_range_exp_inverted_expanded_v2, exp_inverted_loss_expanded_v2_fil, label='Exp Inverted Loss 500', marker='o')

# Añadir títulos y etiquetas
plt.title('Validation Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Mostrar la gráfica
plt.tight_layout()
plt.show()

### Data Explore

In [ ]:
# Ejemplo de matriz de distancias 2x2
distances = np.array([[1000, 8, 2],
                      [1, 3, 90]])

# Llamar a la función calculate_adjacency_matrix con la matriz de distancias de ejemplo
adjacency_matrix = calculate_adjacency_matrix(distances, sigma_squared=10, epsilon=0.5)

# Imprimir la matriz de adyacencia resultante
adjacency_matrix

In [ ]:
def search_value_in_dataframe(df, value):

    result = []

    for col in df.columns:
        matching_rows = df[df[col] == value].index.tolist()
        for row in matching_rows:
            result.append((row, col))
    return result

In [ ]:
data = pd.read_csv(data_path)
data.head()

#### Train

In [ ]:
print(f'x_train size: {x_train_expl.shape}')
print(f'y_train size: {y_train_expl.shape}')

In [ ]:
search_value_in_dataframe(data, x_train_expl[23][0][0][0].item())

In [ ]:
y_train_expl[0][0].item()

In [ ]:
search_value_in_dataframe(data, y_train_expl[0][0].item())